In [5]:
pip install pandas plotly numpy python-dotenv ipywidgets openpyxl

  Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl.metadata (89 kB)
  Using cached ipywidgets-8.1.5-py3-none-any.whl.metadata (2.3 kB)
  Using cached widgetsnbextension-4.0.13-py3-none-any.whl.metadata (1.6 kB)
  Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl.metadata (4.1 kB)
Using cached pandas-2.2.3-cp312-cp312-macosx_11_0_arm64.whl (11.4 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 34.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 32.2 MB/s eta 0:00:00
Using cached ipywidgets-8.1.5-py3-none-any.whl (139 kB)
Using cached jupyterlab_widgets-3.0.13-py3-none-any.whl (214 kB)
Using cached widgetsnbextension-4.0.13-py3-none-any.whl (2.3 MB)


In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import FileUpload, VBox, HBox, FloatSlider, SelectMultiple, Button, Output
from IPython.display import display, clear_output

# Enable plotly to work in notebook
import plotly.io as pio
pio.renderers.default = 'notebook'

False

In [7]:
# Test the file upload and data processing
uploader = FileUpload(
    accept='.xlsx,.xls',
    multiple=False,
    description='Upload Excel File'
)

# Create output area for data preview
preview_output = Output()

def test_file_upload():
    if not uploader.value:
        print("Please upload a file first")
        return
    
    # Get the uploaded file
    file_name = list(uploader.value.keys())[0]
    file_data = uploader.value[file_name]['content']
    
    # Read the Excel file
    df = pd.read_excel(file_data)
    
    # Display basic information about the dataset
    print("Dataset Info:")
    print(f"Number of rows: {len(df)}")
    print(f"Number of columns: {len(df.columns)}")
    print("\nColumns:")
    for col in df.columns:
        print(f"- {col}")
    
    # Display first few rows
    print("\nFirst few rows:")
    display(df.head())
    
    return df

# Run the test
df = test_file_upload()

FileUpload(value=(), accept='.xlsx,.xls', description='Upload')

In [ ]:
# Create widgets for criteria selection and weighting
criteria_output = Output()

def create_criteria_widgets():
    if 'df' not in globals():
        with criteria_output:
            print("Please upload a file first")
        return
    
    # Create criteria selection widget
    criteria_select = SelectMultiple(
        options=df.columns.tolist(),
        description='Select Criteria',
        style={'description_width': 'initial'}
    )
    
    # Create weight sliders container
    weight_sliders = VBox([])
    
    def on_criteria_change(change):
        # Clear existing sliders
        weight_sliders.children = []
        
        # Create new sliders for selected criteria
        for criterion in change['new']:
            slider = FloatSlider(
                value=1.0,
                min=0.0,
                max=5.0,
                step=0.1,
                description=criterion,
                style={'description_width': 'initial'}
            )
            weight_sliders.children += (slider,)
    
    criteria_select.observe(on_criteria_change, names='value')
    
    # Create calculate button
    calculate_button = Button(description='Calculate Scores')
    
    def on_calculate_click(b):
        if not criteria_select.value:
            print("Please select at least one criterion")
            return
            
        # Calculate weighted scores
        total_weight = sum(slider.value for slider in weight_sliders.children)
        weighted_scores = pd.Series(0.0, index=df.index)
        
        for criterion, slider in zip(criteria_select.value, weight_sliders.children):
            # Normalize the criterion scores
            normalized_scores = (df[criterion] - df[criterion].min()) / (df[criterion].max() - df[criterion].min())
            weighted_scores += normalized_scores * (slider.value / total_weight)
        
        # Add scores to dataframe
        df['Composite_Score'] = weighted_scores
        
        # Display results
        print("\nTop 10 Companies by Score:")
        display(df.nlargest(10, 'Composite_Score')[['Composite_Score'] + list(criteria_select.value)])
        
        # Create score distribution plot
        fig = px.histogram(df, x='Composite_Score',
                          title='Distribution of Composite Scores')
        fig.show()
    
    calculate_button.on_click(on_calculate_click)
    
    # Display all widgets
    display(criteria_select)
    display(weight_sliders)
    display(calculate_button)

with criteria_output:
    create_criteria_widgets()

display(criteria_output)

In [ ]:
# Create percentile filter widget
percentile_output = Output()

def create_percentile_filter():
    if 'df' not in globals() or 'Composite_Score' not in df.columns:
        with percentile_output:
            print("Please calculate scores first")
        return
    
    # Create percentile slider
    percentile_slider = FloatSlider(
        value=80,
        min=0,
        max=100,
        step=1,
        description='Score Percentile Threshold',
        style={'description_width': 'initial'}
    )
    
    def on_percentile_change(change):
        threshold = np.percentile(df['Composite_Score'], 100 - change['new'])
        filtered_df = df[df['Composite_Score'] >= threshold]
        
        print(f"\nCompanies above {change['new']}th percentile (score >= {threshold:.3f}):")
        display(filtered_df)
        
        # Create scatter plot of scores
        fig = px.scatter(df, y='Composite_Score',
                         title='Company Scores with Threshold')
        fig.add_hline(y=threshold, line_dash="dash", line_color="red")
        fig.show()
    
    percentile_slider.observe(on_percentile_change, names='value')
    
    # Display widget and initial results
    display(percentile_slider)
    on_percentile_change({'new': percentile_slider.value})

with percentile_output:
    create_percentile_filter()

display(percentile_output)